# Polynomial GCD Dataset — Minimal Example

This notebook shows how **easy** it is to plug a custom *problem generator* into the
`calt` data pipeline.  
Instead of the built‑in `SumProblemGenerator`, we define our own `GCDProblemGenerator`
directly in the notebook, import the rest of the library, and instantly obtain a toy dataset.

## 1. Imports

In [1]:
from typing import Any, List, Tuple, Dict, Union
import random
from sympy import GF, QQ, RR, ZZ
from sympy.polys.rings import ring, PolyRing, PolyElement
from calt.generator.sympy import (
    PolynomialSampler,
    DatasetGenerator,
    BaseStatisticsCalculator,
)

/home/ara_shun/workspace/calt/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Define a Polynomial Ring

In [2]:
# GF(7) with 2 variables x,y
R, *gens = ring("x,y", GF(7), order="grevlex")
R

Polynomial ring in x, y over GF(7) with grevlex order

## 3. Build a Polynomial Sampler

In [3]:
sampler = PolynomialSampler(
    ring=R,
    max_num_terms=5,
    max_degree=10,
    min_degree=1,
    degree_sampling="uniform",  # "uniform" or "fixed"
    term_sampling="uniform",  # "uniform" or "fixed"
    max_coeff=None,  # Used for RR and ZZ
    num_bound=None,  # Used for QQ
    strictly_conditioned=False,
    nonzero_instance=True,
)

### PolynomialSampler's Parameters

#### Basic Parameters

`ring: PolyRing`  
- A SymPy polynomial ring used for generating polynomials  
- Specifies the coefficient field and variables  

`max_num_terms: int = 10`  
- The maximum number of terms in a generated polynomial  
- Default value is 10  

`max_degree: int = 5`  
- The maximum degree of a generated polynomial  
- Default value is 5  

`min_degree: int = 0`  
- The minimum degree of a generated polynomial  
- Default value is 0  

#### Sampling Method Parameters

`degree_sampling: str = "uniform"`  
- Specifies how to sample the degree of a monomial  
- Options:  
  - `"uniform"`: First choose a degree uniformly at random, then choose a monomial of that degree  
  - `"fixed"`: Choose monomials uniformly at random from all possible monomials up to degree `d` (higher-degree monomials are more likely to be chosen)  

`term_sampling: str = "uniform"`  
- Specifies how to sample the number of terms  
- Options:  
  - `"uniform"`: Choose the number of terms uniformly at random between 1 and `max_num_terms`  
  - `"fixed"`: Always generate exactly `max_num_terms` terms  

#### Coefficient Parameters

`max_coeff: Optional[int] = None`  
- The maximum absolute value of coefficients (used for `RR` and `ZZ`)  
- Default value is 10  

`num_bound: Optional[int] = None`  
- The maximum absolute value of the numerator and denominator in rational coefficients (`QQ`)  
- Default value is 10  

#### Constraint Parameters

`strictly_conditioned: bool = True`  
- Whether to generate only polynomials that strictly satisfy the specified conditions  
- When `True`:  
  - Generates only polynomials that exactly match the specified degree and number of terms  
  - Retries generation if the conditions are not met  
- When `False`:  
  - Generates polynomials with relaxed conditions  

`nonzero_instance: bool = False`  
- Whether to generate only non-zero polynomials  
- When `True`: Never generates the zero polynomial  
- When `False`: May generate the zero polynomial  


## 4. Write a **custom** `GCDProblemGenerator`

**Key idea:** The generator is simply a *callable* that returns `(input, target)`.  
If it follows this contract, it can automatically generate samples.

In [4]:
class GCDProblemGenerator:
    """
    Problem generator for polynomial GCD problems.

    This generator creates problems in which the input is a pair of polynomials F = [f_1, f_2],
    and the output is a single polynomial g = GCD(f_1, f_2).
    """

    def __init__(self, sampler: PolynomialSampler):
        """
        Initialize polynomial GCD generator.

        Args:
            sampler: Polynomial sampler
        """
        self.sampler = sampler

    def __call__(self, seed: int) -> Tuple[List[PolyElement], PolyElement]:
        """
        Generate a single sample.

        Each sample consists of:
        - Input polynomial system F
        - Output polynomial g (GCD of F)

        Args:
            seed: Seed for random number generator

        Returns:
            Tuple containing (F, g)
        """
        random.seed(seed)

        # Generate input polynomials using sampler
        base_gcd, q1, q2 = self.sampler.sample(num_samples=3)

        # Generate output polynomial g (GCD of F)
        extra = q1.gcd(q2)
        new_gcd = base_gcd * extra
        q1 = q1.quo(extra) # divide q1 by extra
        q2 = q2.quo(extra) # divide q2 by extra

        F = [new_gcd * q1, new_gcd * q2]
        g = new_gcd.monic()

        return F, g

## 5. Write a **custom** `PolyStatisticsCalculator`

**Key idea:** The calculator is a *callable* that takes `(input, target)` and returns a dictionary of statistics.  
If it follows this contract, it can automatically compute comprehensive statistics for any polynomial system.

<!-- ## Core Functionality
- Takes polynomial inputs and outputs
- Computes detailed statistics about:
  - System size (number of polynomials)
  - Degree distribution (sum/max/min total degrees)
  - Term distribution (max/min terms)
  - Coefficient properties (max/min coefficients)
  - System density


## Key Features
- Handles different coefficient fields (QQ, RR, ZZ, GF)
- Works with both single polynomials and polynomial systems
- Provides comprehensive statistical analysis
- Automatically adapts to different polynomial structures -->

In [5]:
class PolyStatisticsCalculator(BaseStatisticsCalculator):
    """
    Statistics calculator for polynomial problems.
    """

    def __init__(self, ring: PolyRing):
        """
        Initialize polynomial statistics calculator.

        Args:
            ring: Polynomial ring
        """
        self.ring = ring
        self.num_vars = ring.ngens
        self.coeff_field = ring.domain

    def __call__(
        self,
        input: Union[List[PolyElement], PolyElement],
        target: Union[List[PolyElement], PolyElement],
    ) -> Dict[str, Any]:
        """
        Calculate statistics for a single generated sample.

        Args:
            input: Problem (a list of polynomials or a single polynomial)
            target: Solution (a list of polynomials or a single polynomial)

        Returns:
            Dictionary containing statistics about the sample
        """

        if isinstance(input, list):
            input_stats = self.poly_system_stats(input)
        else:
            input_stats = self.poly_system_stats([input])
        if isinstance(target, list):
            output_stats = self.poly_system_stats(target)
        else:
            output_stats = self.poly_system_stats([target])

        return {
            "input": input_stats,
            "output": output_stats,
        }

    def poly_system_stats(self, polys: List[PolyElement]) -> Dict[str, Any]:
        """
        Calculate statistics for a list of polynomials.

        Args:
            polys: List of polynomials

        Returns:
            Dictionary containing statistical information about the polynomials
        """
        num_polys = len(polys) # Number of polynomials in the system

        if num_polys == 0:
            return {"num_polynomials": 0, "total_degree": 0, "total_terms": 0}

        degrees = [self.total_degree(p) for p in polys]
        num_terms = [len(p.terms()) for p in polys]

        coeffs = []
        for p in polys:
            if self.coeff_field == QQ:
                # For QQ, consider both numerators and denominators
                coeffs.extend([abs(float(c.numerator)) for c in p.coeffs()])
                coeffs.extend([abs(float(c.denominator)) for c in p.coeffs()])
            elif self.coeff_field == RR:
                # For RR, take absolute values
                coeffs.extend([abs(float(c)) for c in p.coeffs()])
            elif self.coeff_field == ZZ:
                # For ZZ, take absolute values
                coeffs.extend([abs(int(c)) for c in p.coeffs()])
            elif self.coeff_field.is_FiniteField:  # GF
                # For finite fields, just take the values
                coeffs.extend([int(c) for c in p.coeffs()])

        stats = {
            # System size statistics
            "num_polynomials": num_polys, # Number of polynomials in the system
            # Degree statistics
            "sum_total_degree": sum(degrees), # Sum of total degrees of all polynomials in the system
            "max_total_degree": max(degrees), # Maximum degree of any polynomial in the system
            "min_total_degree": min(degrees), # Minimum degree of any polynomial in the system
            # Term count statistics
            "sum_num_terms": sum(num_terms), # Total number of terms across all polynomials in the system
            "max_num_terms": max(num_terms), # Maximum number of terms in any polynomial in the system
            "min_num_terms": min(num_terms), # Minimum number of terms in any polynomial in the system
            # Coefficient statistics
            "max_abs_coeff": max(coeffs) if coeffs else 0, # Maximum absolute coefficient value in the system
            "min_abs_coeff": min(coeffs) if coeffs else 0, # Minimum absolute coefficient value in the system
            # Additional system properties
            "density": float(sum(num_terms)) / (num_polys * (1 + max(degrees)) ** self.num_vars), # Density of the system (ratio of total terms to maximum possible terms))
        }

        return stats

    def total_degree(self, poly: PolyElement) -> int:
        """Compute total degree of a polynomial.

        The total degree of a polynomial is the maximum sum of exponents among all 
        monomials in the polynomial. For example, in x**2*y + x*y, the total degree
        is 3 (from x**2*y where 2+1=3).
        
        Args:
            poly: Polynomial

        Returns:
            Total degree of the polynomial

        Examples:
            >>> R, x, y = ring("x,y", ZZ)
            >>> calc = PolyStatisticsCalculator(R)
            >>> p = x**2*y + x*y**2 + x + y
            >>> calc.total_degree(p)
            3
        """
        if poly.is_zero:
            return 0
        else:
            return max(list(sum(monom) for monom in poly.monoms()))

## 6. Create Data & Inspect a Sample

In [6]:
# Initialize problem generator
problem_generator = GCDProblemGenerator(sampler=sampler)

# Single sample
F, g = problem_generator(seed=123)
print("Inputs F:", F)
print("Target g:", g)

Inputs F: [6 mod 7*x*y + 5 mod 7*x + 6 mod 7*y + 5 mod 7, 2 mod 7*x**2*y**2 + 2 mod 7*x*y**2 + 5 mod 7*x**2 + 2 mod 7*x + 4 mod 7]
Target g: x + 1 mod 7


## 7. Generate a Tiny Dataset

### Initialize dataset generator

In [7]:
dataset_generator = DatasetGenerator(
    backend="multiprocessing",
    n_jobs=1,  # warning: the current version with Sympy backend only supports n_jobs=1.
    verbose=True, # Whether to print progress.
    root_seed=100, # Seed for random number generator.
)


### Initialize statistics calculator

In [8]:
statistics_calculator = PolyStatisticsCalculator(ring=R)

### Generate training set

In [9]:
samples, stats = dataset_generator.run(
    train=True, # Whether to generate training set.
    num_samples=20, # Number of samples to generate.
    problem_generator=problem_generator, # Problem generator for polynomial GCD problems.
    statistics_calculator=statistics_calculator, # Statistics calculator for polynomial problems.
)

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


### Show first three examples

In [9]:
for i, (F_i, g_i) in enumerate(samples[:3]):
    print(f"--- Sample {i} ---")
    print("F:", F_i)
    print("g:", g_i)

--- Sample 0 ---
F: [3 mod 7*x**7 + 4 mod 7*x**6 + 5 mod 7*x**5*y + x**5 + 6 mod 7*x**3 + 5 mod 7*x**2 + 3 mod 7*x*y + 5 mod 7*x + 2 mod 7*y + 6 mod 7, 4 mod 7*x**5*y**5 + 6 mod 7*x**8*y + 5 mod 7*x**7*y**2 + 2 mod 7*x**7 + x*y**5 + 3 mod 7*x**5 + 5 mod 7*x**4*y + 3 mod 7*x**3*y**2 + 3 mod 7*y**5 + x**3*y + 2 mod 7*x**2*y**2 + 4 mod 7*x**3 + 5 mod 7*x**2 + 6 mod 7*x + 4 mod 7]
g: x**5 + 2 mod 7*x + 6 mod 7
--- Sample 1 ---
F: [4 mod 7*x**7 + 5 mod 7*x**5*y**2 + x**2*y**5 + 3 mod 7*y**7 + 5 mod 7*x**6 + 5 mod 7*x**5*y + 3 mod 7*x*y**5 + 3 mod 7*y**6 + 2 mod 7*x**5 + 4 mod 7*y**5, 2 mod 7*x**8 + x**6*y**2 + 4 mod 7*x**3*y**5 + 2 mod 7*x*y**7 + 6 mod 7*x**5 + 5 mod 7*y**5]
g: x**5 + 2 mod 7*y**5
--- Sample 2 ---
F: [x**3*y**2 + 3 mod 7*x**2*y**2, x*y + 3 mod 7*y]
g: x*y + 3 mod 7*y


### Show statistics

In [12]:
import yaml
print(yaml.dump(stats, sort_keys=False))

total_time: 0.008996963500976562
samples_per_second: 2222.9722281110876
num_samples: 20
generation_time:
  mean: 0.00040371417999267577
  std: 0.0004956771003782554
  min: 8.034706115722656e-05
  max: 0.002209901809692383
input_overall:
  num_polynomials:
    mean: 2.0
    std: 0.0
    min: 2.0
    max: 2.0
  sum_total_degree:
    mean: 14.05
    std: 5.8006465156911595
    min: 5.0
    max: 29.0
  max_total_degree:
    mean: 8.35
    std: 3.4967842369811724
    min: 3.0
    max: 17.0
  min_total_degree:
    mean: 5.7
    std: 2.6095976701399777
    min: 2.0
    max: 12.0
  sum_num_terms:
    mean: 14.35
    std: 8.266045003506816
    min: 4.0
    max: 32.0
  max_num_terms:
    mean: 9.25
    std: 5.855552920092175
    min: 2.0
    max: 24.0
  min_num_terms:
    mean: 5.1
    std: 3.160696125855822
    min: 1.0
    max: 12.0
  max_abs_coeff:
    mean: 5.7
    std: 0.7810249675906654
    min: 3.0
    max: 6.0
  min_abs_coeff:
    mean: 1.2
    std: 0.5099019513592785
    min: 1.0
    ma